# **使用 Python 進行管道 ETL**

---


此程式碼旨在舉例說明使用 Python 的 ETL(Extract, Transform, Load)管道。程式碼分為三個步驟：

1) 透過網頁抓取提取多個資料；
2) 數據轉換；
3) 將已經結構化的資料載入資料庫；

主要的自動化指令：

* 檢查參考資料夾中的文件，選擇已有資料的日期；
* 下載沒有資料的日期（txt格式但壓縮）；
* 解壓縮 zip 文件，然後將其刪除；
* 載入txt文件，將其轉換為parquet並刪除txt檔案；
* 更正資料格式（str、float 和 datetime）並刪除冗餘列；
* 儲存檔案並執行表格之間的資料完整性測試；
* 首次載入本機 SQL 資料庫；
* 對已建立的資料庫進行第二次（增量）載入。



ETL:

ETL是Extract、Transform、Load的縮寫，就是提取、轉換和載入。 ETL 是資料工程中的基本流程，廣泛應用於商業智慧 (BI) 和資料倉儲環境中，用於將多個來源的資料整合到單一儲存庫中，從而促進分析和決策。
---

## 1. 提取

### 1.1.下載和安裝需要的庫

In [1]:
!pip install wget mplfinance gdown

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.1 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=75a8235533ab54313044e19214e0480cdd801fe1abf43433cde51b6187b336bf
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [2]:
import os
import wget
import httplib2
import shutil
import zipfile
import pyarrow
import numpy as np
import pandas as pd
import datetime
import sqlite3
import mplfinance as mpf
import plotly.graph_objects as go

### 1.2.數據來源

資料來源是 ETL 管道的起點。這些可以多種多樣，包括資料庫、雲端儲存、應用程式日誌、外部 API 等。

在本個範例中，我們將提取：

* B3 每日交易資料。這種「逐筆」數據顯示了所進行的每筆交易，包括數量、價格和涉及的代理商。 B3 免費提供過去 20 個工作天的歷史資料。

https://www.b3.com.br/pt_br/market-data-e-indices/servicos-de-dados/market-data/cotacoes/cotacoes/

* 此數據中包含的資訊的詞彙表：

https://www.b3.com.br/data/files/14/42/28/31/FEC4A8103234E0A8AC094EA8/Glossario_NegociosListados_PT.pdf

* B3參與者的數據

https://sistemaswebb3-listados.b3.com.br/participantsPage/

### 1.3.萃取

提取是 ETL 過程的第一步。它涉及從各種來源獲取資料並將其儲存在集中位置以進行處理。提取方法各不相同，包括批次（預定間隔）或即時串流處理


功能 download_b3_tick_by_tick

1. 檢查哪些文件已經下載；
2. 過濾該月的日期
3. 檢查網站上是否存在該日期
4. 如果該日期存在，請下載 .zip 文件
5. 刪除執行此函數時所建立的“.cache”資料夾。

In [3]:
def download_b3_tick_by_tick(start=datetime.datetime(2024,1,1)):

    folder_ref = os.getcwd()
    files = os.listdir(folder_ref)
    downloaded_files_zip = [i for i in files if i.endswith('.zip')]
    downloaded_dates = [i.replace('B3_tick_', '').replace('.zip', '') for i in downloaded_files_zip]
    downloaded_dates = pd.to_datetime(downloaded_dates)

    date_range = pd.date_range(start.date(), datetime.datetime.now().date())
    date_range_to_download = date_range[~date_range.isin(downloaded_dates)]

    for i in date_range_to_download:

        try:
            market_date = i.strftime('%Y-%m-%d')
            url = f'https://arquivos.b3.com.br/apinegocios/tickercsv/{market_date}'
            h = httplib2.Http('.cache')
            response, content = h.request(f'https://arquivos.b3.com.br/apinegocios/tickercsv/{market_date}', headers={'Connection':'keep-alive'})
            if len(content)>0:
                response = wget.download(url, str('B3_tick_'+market_date+'.zip'))

        except:
            None

    shutil.rmtree('.cache')



unzip_files_delete 函數

1. 檢查資料夾內的文件；
2. 對於資料夾中的每個文件，如果以“.zip”結尾，則解壓縮；
3. 對於資料夾中的每個文件，如果以“.zip”結尾，則將其刪除；

In [4]:
def unzip_files_delete():

    folder_ref = os.getcwd()
    files = os.listdir(folder_ref)

    for i in files:
        if i.endswith('.zip'):
            with zipfile.ZipFile(i, "r") as zip_ref:
                zip_ref.extractall(folder_ref)

    for i in files:
        if i.endswith('.zip'):
            os.remove(i)

讀取B3參與者的csv檔案（必須在資料夾中）

In [5]:
import pandas as pd
import requests

csv_url = 'https://raw.githubusercontent.com/markl-a/My-AI-Learning-Notes/main/1.%E5%BE%9EAI%E5%88%B0LLM%E5%9F%BA%E7%A4%8E/1.AI%2CML/participantDownload.csv'


# Download the file
response = requests.get(csv_url)
with open('participantDownload.csv', 'wb') as f:
    f.write(response.content)

codigo_participantes_B3 = pd.read_csv("participantDownload.csv", sep=';')

執行已建立的兩個函數

In [6]:
download_b3_tick_by_tick(start=datetime.datetime(2024,7,1))
unzip_files_delete()

## 2、轉換

從提取中獲得的原始數據很少適合分析。轉換階段涉及清理、豐富和結構化數據，使其可用於預期目的。這可能包括資料驗證、資料類型轉換、重複資料刪除和來自不同來源的資料聚合等任務。

### 2.1.將 txt 資料轉換為 parquet（節省磁碟機空間 > 70%）

Convert_to_parquet 函數：

1. 列出資料夾內的txt檔案（每日交易資料）；
2. 將每個 txt 檔案作為資料幀加載；
3. 以parquet格式匯出，優化文件名為年/月/日格式；
4. 刪除txt檔案；

In [7]:
import gdown

file_id = '17-dIkpVkBypZsarY319ybx9gkAEksr4M'
gdown.download(id=file_id, output="04-09-2024_NEGOCIOSAVISTA.txt", quiet=False)

folder_ref = os.getcwd()
files = os.listdir(folder_ref)
files_txt = [i for i in files if i.endswith('.txt')]

for i in files_txt:
    df = pd.read_csv(i, sep=";")
    df.to_parquet('raw_'+str(i[6:10] + '_' + i[3:5] + '_' + i[0:2] + '_' + 'tick_B3.parquet'))

for i in files_txt:
    os.remove(i)

Downloading...
From (original): https://drive.google.com/uc?id=17-dIkpVkBypZsarY319ybx9gkAEksr4M
From (redirected): https://drive.google.com/uc?id=17-dIkpVkBypZsarY319ybx9gkAEksr4M&confirm=t&uuid=fb720c58-66a7-49e1-b84b-755ec12459bf
To: /content/04-09-2024_NEGOCIOSAVISTA.txt
100%|██████████| 620M/620M [00:07<00:00, 82.5MB/s]


### 2.2.探索資料中的“問題”

此步驟通常不會進入管道，因為通常資料工程師已經找到了潛在的資料問題來清理、豐富和結構化資料。

但為了更好地理解，讓我們先閱讀一份每日交易文件，以了解一些資料

### 2.2.1.每日交易資料

In [8]:
df_tick_dia1 = pd.read_parquet('raw_2024_09_04_tick_B3.parquet', engine='pyarrow')
df_tick_dia1

,DataReferencia,CodigoInstrumento,AcaoAtualizacao,PrecoNegocio,QuantidadeNegociada,HoraFechamento,CodigoIdentificadorNegocio,TipoSessaoPregao,DataNegocio,CodigoParticipanteComprador,CodigoParticipanteVendedor
0,2024-09-04,TF583R,0,"10,000",10000,42246704,10,1,2024-09-04,100.0,100.0
1,2024-09-04,WINZ24,0,"138285,000",1,90000003,10,1,2024-09-04,3.0,93.0
2,2024-09-04,WINZ24,0,"138285,000",1,90000003,20,1,2024-09-04,85.0,85.0
3,2024-09-04,WINZ24,0,"138285,000",1,90000003,30,1,2024-09-04,85.0,3.0
4,2024-09-04,WSPU24,0,"5520,000",1,90000007,10,1,2024-09-04,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...
9063754,2024-09-04,INDV24,0,"137995,000",5,183127121,144000,1,2024-09-04,16.0,93.0
9063755,2024-09-04,INDV24,0,"137995,000",5,183127121,144010,1,2024-09-04,16.0,308.0
9063756,2024-09-04,DI1V24,0,"10,526",15000,183519338,31445,1,2024-09-04,107.0,107.0
9063757,2024-09-04,DIIF26F27,2,"0,010",840,184924833,250,1,2024-09-04,NaN,NaN


我們要在 900 萬行和 11 列的資料中找到一些特徵和潛在問題

#### 2.2.1.1.問題：資料幀中的資料類型（Dtype）

In [9]:
df_tick_dia1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9063759 entries, 0 to 9063758
Data columns (total 11 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   DataReferencia               object 
 1   CodigoInstrumento            object 
 2   AcaoAtualizacao              int64  
 3   PrecoNegocio                 object 
 4   QuantidadeNegociada          int64  
 5   HoraFechamento               int64  
 6   CodigoIdentificadorNegocio   int64  
 7   TipoSessaoPregao             int64  
 8   DataNegocio                  object 
 9   CodigoParticipanteComprador  float64
 10  CodigoParticipanteVendedor   float64
dtypes: float64(2), int64(5), object(4)
memory usage: 760.7+ MB


#### 2.2.1.2。問題：“PrecoNegocio”(BusinessPrice)列的小數除數為“逗號”

In [10]:
df_tick_dia1.head()

,DataReferencia,CodigoInstrumento,AcaoAtualizacao,PrecoNegocio,QuantidadeNegociada,HoraFechamento,CodigoIdentificadorNegocio,TipoSessaoPregao,DataNegocio,CodigoParticipanteComprador,CodigoParticipanteVendedor
0,2024-09-04,TF583R,0,"10,000",10000,42246704,10,1,2024-09-04,100.0,100.0
1,2024-09-04,WINZ24,0,"138285,000",1,90000003,10,1,2024-09-04,3.0,93.0
2,2024-09-04,WINZ24,0,"138285,000",1,90000003,20,1,2024-09-04,85.0,85.0
3,2024-09-04,WINZ24,0,"138285,000",1,90000003,30,1,2024-09-04,85.0,3.0
4,2024-09-04,WSPU24,0,"5520,000",1,90000007,10,1,2024-09-04,3.0,3.0


#### 2.2.1.3。問題：列包含冗餘或不太有用的信息

「AcaoAtualizacao」欄告知交易是否被取消。預設值為 0，2 表示已取消交易。在下一個輸出中看到，取消的交易很少見，但無論如何我們都會在清理階段刪除 AcaoAtualizacao=2 的行

In [11]:
df_tick_dia1[df_tick_dia1.AcaoAtualizacao!=0]

,DataReferencia,CodigoInstrumento,AcaoAtualizacao,PrecoNegocio,QuantidadeNegociada,HoraFechamento,CodigoIdentificadorNegocio,TipoSessaoPregao,DataNegocio,CodigoParticipanteComprador,CodigoParticipanteVendedor
2306949,2024-09-04,FRP0,2,"14,800",230,104310587,1210,1,2024-09-04,NaN,NaN
4940808,2024-09-04,PETRS34,2,"0,930",5000000,124846452,10,1,2024-09-04,NaN,NaN
5136694,2024-09-04,DAPK29,2,"6,320",290,130337788,200,1,2024-09-04,NaN,NaN
5418758,2024-09-04,ALOSA206,2,"4,000",380000,132444656,10,1,2024-09-04,NaN,NaN
5868357,2024-09-04,RRRP3T,2,"26,040",12339,140353999,210,1,2024-09-04,NaN,NaN
5868358,2024-09-04,RRRP3T,2,"26,050",27661,140353999,220,1,2024-09-04,NaN,NaN
5951752,2024-09-04,CPMU24C100250,2,"57,000",100,141019716,120,1,2024-09-04,NaN,NaN
6610780,2024-09-04,DOLV24,2,"5650,000",115,145523070,171960,1,2024-09-04,NaN,NaN
7971106,2024-09-04,WDOV24,2,"5647,000",500,161958272,6267390,1,2024-09-04,NaN,NaN
8208795,2024-09-04,IBOVJ139,2,"2095,000",170,163308417,200,1,2024-09-04,NaN,NaN


請注意，下面的「DataReferencia」和「DataNegocio」列差別很小，僅針對商品資產且在少數情況下。透過僅保留“DataReferencia”來消除這種冗餘。其他列（例如“CodigoIdentificadorNegocio”和“TipoSessaoPregao”）不會為我們的分析添加有用信息，因此將被刪除

In [12]:
print('"DataReferencia" difere da "DataNegocio" em apenas', (df_tick_dia1.DataReferencia != df_tick_dia1.DataNegocio).sum(), 'linhas')
set(df_tick_dia1[df_tick_dia1.DataReferencia != df_tick_dia1.DataNegocio]['CodigoInstrumento'])

"DataReferencia" difere da "DataNegocio" em apenas 301 linhas


{'BGIU24',
 'BGIV24',
 'BGIX24',
 'BGIZ24',
 'CCMF25',
 'CCMU24',
 'CCMU25',
 'CCMX24',
 'CCMX25',
 'ETHX24',
 'FRP1',
 'ICFH25',
 'ICFZ24'}

#### 2.2.1.4。問題：HHMMSSNNN 中的「ClosingTime」列

資料為整數，因此上午 10 點之前的時間少一個數字。毫秒也無法為本研究提供有用的資訊。

In [13]:
df_tick_dia1['HoraFechamento']

,HoraFechamento
0,42246704
1,90000003
2,90000003
3,90000003
4,90000007
...,...
9063754,183127121
9063755,183127121
9063756,183519338
9063757,184924833


### 2.2.2.參與者資料

在此數據中，我們僅查找“名稱”和“代碼”列。名稱表示 B3 參與者，代碼指協商資料（列「CodigoParticipanteBuyer」、「CodigoParticipanteVendor」）

請注意，即使在同一單元內，同一代理程式也有多個代碼。我們只需要有「完整交易參與者」資料的參與者

In [14]:
codigo_participantes_B3

,CNPJ,Nome,Site,Perfil,Código
0,191,BANCO DO BRASIL S/A,www.bb.com.br,ADMINISTRADOR iMERCADO,1124 - 2659
1,191,BANCO DO BRASIL S/A,www.bb.com.br,AGENTE DE CUSTÓDIA,254 - 705 - 1123 - 1124 - 1126 - 1345 - 2659 -...
2,191,BANCO DO BRASIL S/A,www.bb.com.br,CUSTODIANTE BALCÃO,254 - 1123
3,191,BANCO DO BRASIL S/A,www.bb.com.br,EMITENTE DE GARANTIAS,10001
4,191,BANCO DO BRASIL S/A,www.bb.com.br,ESCRITURADOR,6098
...,...,...,...,...,...
29134,97548167000123,JGP STRATEGY MASTER FIM,NaN,SEGMENTO BALCAO,13491.00-8
29135,97837181000147,DEXCO S.A,NaN,SEGMENTO BALCAO,26320.40-3
29136,97929213000134,WHG SISTEMA II FUNDO DE INVESTIMENTO MULTIMERC...,NaN,SEGMENTO BALCAO,13565.00-6
29137,98102924000101,DELTASUL UTILIDADES LTDA,NaN,SEGMENTO BALCAO,42836.40-7


In [42]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=codigo_participantes_B3)

https://docs.google.com/spreadsheets/d/1RNr4PCS6ubw_NMf0slNGqb9xx7S8uWRXtkfxJKP2JGs#gid=0


/usr/local/lib/python3.10/dist-packages/google/colab/sheets.py:31: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return frame.applymap(_clean_val).replace({np.nan: None})


### 2.3. 資料清洗與增強

#### 2.3.1.交易資料

Change_dtypes函數：

1. 將「DataReferencia」列轉成datetime格式；
2. 將“CodigoParticipanteComprador”和“CodigoParticipanteVendedor”列中的 NaN 替換為 0；
3. 將欄位「CodigoParticipanteComprador」和「CodigoParticipanteVendedor」轉換為 int（刪除小數位），然後轉換為 str；
4. 將“PrecoNegocio”列轉乘 float；

In [15]:
df_tick_dia1

,DataReferencia,CodigoInstrumento,AcaoAtualizacao,PrecoNegocio,QuantidadeNegociada,HoraFechamento,CodigoIdentificadorNegocio,TipoSessaoPregao,DataNegocio,CodigoParticipanteComprador,CodigoParticipanteVendedor
0,2024-09-04,TF583R,0,"10,000",10000,42246704,10,1,2024-09-04,100.0,100.0
1,2024-09-04,WINZ24,0,"138285,000",1,90000003,10,1,2024-09-04,3.0,93.0
2,2024-09-04,WINZ24,0,"138285,000",1,90000003,20,1,2024-09-04,85.0,85.0
3,2024-09-04,WINZ24,0,"138285,000",1,90000003,30,1,2024-09-04,85.0,3.0
4,2024-09-04,WSPU24,0,"5520,000",1,90000007,10,1,2024-09-04,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...
9063754,2024-09-04,INDV24,0,"137995,000",5,183127121,144000,1,2024-09-04,16.0,93.0
9063755,2024-09-04,INDV24,0,"137995,000",5,183127121,144010,1,2024-09-04,16.0,308.0
9063756,2024-09-04,DI1V24,0,"10,526",15000,183519338,31445,1,2024-09-04,107.0,107.0
9063757,2024-09-04,DIIF26F27,2,"0,010",840,184924833,250,1,2024-09-04,NaN,NaN


In [16]:
def change_dtypes(df):
    df['DataReferencia'] = pd.to_datetime(df['DataReferencia'])
    df[['CodigoParticipanteComprador','CodigoParticipanteVendedor']] = df[['CodigoParticipanteComprador','CodigoParticipanteVendedor']].fillna(0)
    df[['CodigoParticipanteComprador','CodigoParticipanteVendedor']] = df[['CodigoParticipanteComprador','CodigoParticipanteVendedor']].astype('int').astype('str')
    df['PrecoNegocio'] = df.PrecoNegocio.str.replace(",", ".").astype('float')

1. 將「HoraFechamento」列轉換為 str 並在開頭以零補全，以便它們都有 9 個字元；
2. 刪除最後 3 個字元（毫秒）
3. 時間變換 (H:M:S)

In [17]:
def change_HoraFechamento(df):
    df['HoraFechamento'] = df['HoraFechamento'].astype(str).str.zfill(9)
    df['HoraFechamento'] = df['HoraFechamento'].str[:-3]
    df['HoraFechamento'] = pd.to_datetime(df['HoraFechamento'], format='%H%M%S').dt.time

刪除無用資料函數：

1. 代表已取消交易的過濾器行 ("AcaoAtualizacao"=2)；
2. 僅保留您想要的列：

    * "DataReferencia",
    * "CodigoInstrumento",
    * "PrecoNegocio",
    * "QuantidadeNegociada",
    * "HoraFechamento",
    * "CodigoParticipanteComprador",
    * "CodigoParticipanteVendedor"

In [18]:
def remove_useless_data(df):
    df = df[df.AcaoAtualizacao!=2]
    df = df[['DataReferencia', 'CodigoInstrumento', 'PrecoNegocio',
             'QuantidadeNegociada', 'HoraFechamento',
             'CodigoParticipanteComprador', 'CodigoParticipanteVendedor']]
    return df

clean_tick_data_parquet 函數：

1. 從資料夾中載入所有鑲木地板檔案作為資料框；
2. 執行“change_dtypes”函數；
3. 執行“change_HoraFechamento”函數；
4. 執行“remove_useless_data”函數，該函數會傳回一個新的資料幀；
5. 儲存包含所有修改的新鑲木地板文件，重命名時不以「raw_」開頭；
6. 刪除原始資料鑲木地板文件

In [19]:
def cleaning_tick_data_parquet():
    folder_ref = os.getcwd()
    files = os.listdir(folder_ref)
    files_parquet = [i for i in files if i.endswith('.parquet') & i.startswith('raw_')]

    for i in files_parquet:
        df = pd.read_parquet(i, engine='pyarrow')
        change_dtypes(df)
        change_HoraFechamento(df)
        cleaned_df = remove_useless_data(df)
        cleaned_df.to_parquet(i[4:])
        os.remove(i)

In [20]:
cleaning_tick_data_parquet()

#### 2.3.2.參與者資料

clean_participants函數：

1. 在「個人資料」欄中過濾「完整交易參與者」；
2. 維護「名稱」和「代碼」欄；
3. 僅保留參與者的第一個代碼（很少有人保留以 - 分隔的兩個代碼）；
4. 包括最後一行，名稱 =“NOT IDENTIFIED”且代碼 =“0”
5. 將“代碼”列重命名為“Participant_index”
6. 重置索引

In [21]:
def cleaning_participantes(df):
    df = df[df.Perfil=='PARTICIPANTE DE NEGOCIAÇÃO PLENO']
    df = df[['Nome', 'Código']]
    cleaned_index = codigo_participantes_B3['Código'].str.split(' -', n=1, expand=True)[0]
    df['Código'] = cleaned_index
    df.loc[len(df)] = ["NAO IDENTIFICADO", '0']
    df = df.rename(columns={"Código": "Participante_index"})
    df = df.reset_index(drop=True)
    df.to_parquet('cleaned_participantes_index.parquet')

In [22]:
cleaning_participantes(codigo_participantes_B3)

### 2.4. 資料庫之間的完整性、驗證和聚合測試

確保資料實際上已準備好載入非常重要。這些測試對於確保要載入的資料能夠以高效可靠的方式進行分析至關重要。例如，我們測試一下日常交易資料集是否可以與 B3 參與者資料集相關聯

In [23]:
participantes_B3 = pd.read_parquet('cleaned_participantes_index.parquet', engine='pyarrow')
df_tick_dia1 = pd.read_parquet('2024_09_04_tick_B3.parquet', engine='pyarrow')

merged_df = df_tick_dia1[['DataReferencia',
                          'CodigoInstrumento',
                          'CodigoParticipanteComprador']].merge(participantes_B3,
                                                                how='left',
                                                                left_on='CodigoParticipanteComprador',
                                                                right_on='Participante_index')

merged_df

,DataReferencia,CodigoInstrumento,CodigoParticipanteComprador,Nome,Participante_index
0,2024-09-04,TF583R,100,NaN,NaN
1,2024-09-04,WINZ24,3,XP INVESTIMENTOS CCTVM S/A,3
2,2024-09-04,WINZ24,85,BTG PACTUAL CTVM S/A,85
3,2024-09-04,WINZ24,85,BTG PACTUAL CTVM S/A,85
4,2024-09-04,WSPU24,3,XP INVESTIMENTOS CCTVM S/A,3
...,...,...,...,...,...
9063735,2024-09-04,INDV24,93,NOVA FUTURA CTVM LTDA,93
9063736,2024-09-04,INDV24,16,J.P. MORGAN CCVM S/A,16
9063737,2024-09-04,INDV24,16,J.P. MORGAN CCVM S/A,16
9063738,2024-09-04,DI1V24,107,TERRA INVESTIMENTOS DTVM LTDA,107


## 3. 加載

ETL 過程中的載入步驟是最後階段，轉換後的資料將載入到最終目的地。此目的地可以是資料倉儲、資料湖、專用資料庫或任何其他用於分析和報告的資料儲存結構。

了解載入方法至關重要。例如，如果是資料的完全加載，則所有資料都會立即加載到目標中，這將覆蓋任何現有資料。它主要用於第一次載入或在後續的資料更新不可行時。在後續資料載入中，僅載入自上次載入以來新的或修改的資料。

### 3.1.連接到資料庫

在此範例中將建立一個本機 SQL 資料庫，即在我們執行程式碼的資料夾中。這樣，它就不會是雲端資料庫（可能需要儲存和執行成本）。例如，此資料夾可以是 Intranet 上的共用資料夾，以允許其他使用者也探索資料。

In [24]:
connection = sqlite3.connect('database_negociacoes_B3.db')
cursor = connection.cursor()

請注意，在您的資料夾中建立了一個“資料庫檔案 (.db)”，但檔案大小為 0。

### 3.2.初始資料庫加載

SQL 資料庫將由兩個表組成：participants_B3 和 negotiation_diarias。

### 3.2.1.參與者資料

對於參與者_B3資料表是一個僅在第一個例程中載入一次的表。我們將載入parquet檔案作為資料框並將導入到database_negociacoes_B3中。 “database_negociacoes_B3”.db 中該表的名稱將為「Participantes_B3」。

In [25]:
df_participantes_B3 = pd.read_parquet('cleaned_participantes_index.parquet', engine='pyarrow')
df_participantes_B3.to_sql(name='Participantes_B3', con=connection, index=False)

93

現在可以看到database_negociacoes_B3.db 檔案的大小不再為0

### 3.2.1.1.測試對資料庫的查詢

這裡我們需要了解一點SQL語言。

讓我們進行三個快速查詢：

1. 取得表中前5條數據
2. 取得索引為「16」的參與者的姓名

In [26]:
query = """
SELECT *
FROM Participantes_B3 LIMIT 5
"""

cursor.execute(query)
cursor.fetchall()

[('BANCO DO BRASIL S/A', '20017'),
 ('BANCO CENTRAL DO BRASIL', '810'),
 ('BANCO B3 S.A.', '500'),
 ('BANCO RABOBANK INTERNATIONAL BRASIL S/A', '50054'),
 ('BANCO COOPERATIVO SICREDI S/A', '50087')]

In [27]:
query = """
SELECT *
FROM Participantes_B3
WHERE Participante_index = '16'
"""

cursor.execute(query)
cursor.fetchone()

('J.P. MORGAN CCVM S/A', '16')

### 3.2.2.交易資料

#### 3.2.2.1. 交易資料的完全載入

出於教學目的，我們假設我們只有提取和轉換載入前三天的交易資料。

讓我們將 parquet 檔案作為資料幀加載，連接它們並將它們導入在我們的資料庫_negociacoes_B3 中，表名稱為“Negociacoes_diarias”

In [28]:
negociacoes_dia1 = pd.read_parquet('2024_09_04_tick_B3.parquet', engine='pyarrow')
negociacoes_dia2 = pd.read_parquet('2024_09_04_tick_B3.parquet', engine='pyarrow')
negociacoes_dia3 = pd.read_parquet('2024_09_04_tick_B3.parquet', engine='pyarrow')

In [29]:
negociacoes = pd.concat([negociacoes_dia1, negociacoes_dia2, negociacoes_dia3], ignore_index=True)
negociacoes

,DataReferencia,CodigoInstrumento,PrecoNegocio,QuantidadeNegociada,HoraFechamento,CodigoParticipanteComprador,CodigoParticipanteVendedor
0,2024-09-04,TF583R,10.000,10000,04:22:46,100,100
1,2024-09-04,WINZ24,138285.000,1,09:00:00,3,93
2,2024-09-04,WINZ24,138285.000,1,09:00:00,85,85
3,2024-09-04,WINZ24,138285.000,1,09:00:00,85,3
4,2024-09-04,WSPU24,5520.000,1,09:00:00,3,3
...,...,...,...,...,...,...,...
27191215,2024-09-04,INDV24,137995.000,5,18:31:27,93,93
27191216,2024-09-04,INDV24,137995.000,5,18:31:27,16,93
27191217,2024-09-04,INDV24,137995.000,5,18:31:27,16,308
27191218,2024-09-04,DI1V24,10.526,15000,18:35:19,107,107


請注意，我們已經討論了 3000 萬行。您只想堅持使用 Excel 嗎？ ;)

現在該數據已完成加載

In [30]:
negociacoes.to_sql(name='Negociacoes_diarias', con=connection, index=False)

27191220

### 3.3. 關閉與資料庫的連接

In [31]:
connection.close()

### 3.4. 增加資料到 database

這邊是示範把第四天的資料通過ETL導入資料庫

首先讓我們重新建立Python-資料庫連接

In [32]:
connection = sqlite3.connect('database_negociacoes_B3.db')
cursor = connection.cursor()

加載第四天的資料

In [34]:
negociacoes_dia4 = pd.read_parquet('2024_09_04_tick_B3.parquet', engine='pyarrow')
negociacoes_dia4.to_sql(name='Tabela_incremental', con=connection, index=False)

9063740

請注意，我們現在資料庫中有 3 個表

In [35]:
print(pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", connection))

                  name
0     Participantes_B3
1  Negociacoes_diarias
2   Tabela_incremental


讓我們透過查詢將兩者結合。首先我們檢查一下兩個表的大小

In [36]:
query = """
SELECT count('DataReferencia')
FROM Negociacoes_diarias
"""

cursor.execute(query)
cursor.fetchall()

[(27191220,)]

請注意，我們有接近 3000 萬行的資料。現在對 Incremental_Table 進行相同的查詢

In [37]:
query = """
SELECT count('DataReferencia')
FROM Tabela_incremental
"""

cursor.execute(query)

df_participantes = cursor.fetchall()
print(df_participantes)

[(9063740,)]


這有大約九百萬行，現在我們可以透過直接地將兩個表結合在一起



In [38]:
query = """
INSERT INTO Negociacoes_diarias
SELECT *
FROM Tabela_incremental;
"""

cursor.execute(query)

現在可以看到表格已經串聯起來

In [39]:
query = """
SELECT count('DataReferencia')
FROM Negociacoes_diarias
"""

cursor.execute(query)
cursor.fetchall()

[(36254960,)]

最後，我們便可從資料庫中刪除 Incremental_Table 並關閉連接

In [40]:
print(pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", connection))

                  name
0     Participantes_B3
1  Negociacoes_diarias
2   Tabela_incremental


In [41]:
query = """
DROP TABLE Tabela_incremental
"""

cursor.execute(query)

In [43]:
print(pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", connection))

                  name
0     Participantes_B3
1  Negociacoes_diarias


In [44]:
connection.commit()
connection.close()